In [1]:
import os
os.chdir("..")

In [2]:
from src.train_predict_jobs import train, predict
from datetime import datetime

today_date = datetime.today().strftime('%Y-%m-%d')

C:\Users\yshim\Documents\libercraft\遊技機新台スコア予測\consulting_perva-redesign\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json
with open('newest_version_and_current_paths.json', "r") as file:
    paths = json.load(file)

In [4]:
# 最新データで学習を行う
#df_train_test = train(today_date,
#                      hyperparameter_tuning=False, # ハイパーパラメータのチューニングは行わない
#                      dev_database=False) # 新しいデータベースを使用

In [5]:
import pandas as pd
df_train = pd.read_csv(paths['train_data'])
df_test = pd.read_csv(paths['test_data'])

df_train_test = pd.concat([df_train, df_test]).reset_index(drop=True)

In [6]:
# 予測結果の取得
df_output, df_profit_output, df = predict(today_date,
                                          dev_database=False) # 最新のデータベースを使用

2024-11-29 03:49:01,196| ERROR   | Password is required for key C:\Users\yshim/.ssh\id_rsa


SSHトンネルが 59674 にバインドされました。
SSHトンネルを閉じました。


In [7]:
df_output.head(4)

,KatashikiId,Kind,Performance_pred,Spec_pred,Content_pred,News_pred,Concept_pred,Period_pred,SaleUnit_eval_pred,Running_pred,Returns_pred,Price_eval_pred
0,S2019102418435101,1,2.45,2.35,2.50,2.15,3.05,2.85,2.90,1.70,1.35,2.60
1,S2021040810102201,1,2.05,2.35,2.15,2.00,2.80,2.65,3.25,1.95,1.95,3.75
2,P2019100212264701,0,2.05,2.25,2.25,2.05,2.60,2.95,2.65,1.35,1.50,2.60
3,P2020030410274101,0,2.10,2.40,2.45,2.25,2.75,3.00,2.70,1.50,1.55,2.40


In [8]:
df_profit_output.head(4)

,KatashikiId,Kind,GrossProfit_pred,ModelLife_pred
0,S2019102418435101,1,106369.160957,5.463404
1,S2021040810102201,1,94672.267234,6.209293
2,P2019100212264701,0,346716.981113,8.228389
3,P2020030410274101,0,853699.826153,15.122067


In [9]:
# 特徴量を含むデータ
df.head(5)

,KatashikiId,Regist_Date,ReleaseDate,MakerId,Kind,Name,Price,TypeId,TypeDetailId,KakuhenRate1_1,...,Returns_all_past_mean_per_brand_name,Price_eval_90days_mean_per_brand_name,Price_eval_365days_mean_per_brand_name,Price_eval_all_past_mean_per_brand_name,GrossProfit_90days_mean_per_brand_name,GrossProfit_365days_mean_per_brand_name,GrossProfit_all_past_mean_per_brand_name,ModelLife_90days_mean_per_brand_name,ModelLife_365days_mean_per_brand_name,ModelLife_all_past_mean_per_brand_name
0,S2019102418435101,2019-07-26 18:43:51,2020-01-05,54.0,1,Sパチスロ1000ちゃん PX,448000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S2021040810102201,2021-01-08 10:10:22,2021-04-04,54.0,1,Sパチスロ1000ちゃん G1,0,NaN,NaN,NaN,...,1.5,NaN,NaN,2.5,NaN,NaN,133348.0,NaN,NaN,5.0
2,P2019100212264701,2019-07-04 12:26:47,2019-12-01,68.0,0,P13日の金曜日 ZB,439000,6.0,7.0,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P2020030410274101,2019-12-05 10:27:41,2020-05-06,68.0,0,P13日の金曜日 MA,439000,3.0,7.0,50.0,...,1.0,NaN,3.0,3.0,NaN,389976.0,389976.0,NaN,6.0,6.0
4,P2020030410381601,2019-12-05 10:38:16,2020-05-06,68.0,0,P13日の金曜日 GLA,439000,4.0,7.0,50.0,...,1.0,NaN,3.0,3.0,NaN,389976.0,389976.0,NaN,6.0,6.0


In [10]:
# データ区分の作成
df['データ区分'] = None
df.loc[df['KatashikiId'].isin(df_train_test['KatashikiId'].tolist()), 'データ区分'] = '学習'
df.loc[df['データ区分'].isnull(), 'データ区分'] = '学習対象外'

In [11]:
import pandas as pd
df_output = pd.merge(df_output, df_profit_output.drop(columns=['Kind']), on='KatashikiId', how='inner')

In [12]:
from data_columns.features_and_targets import features, targets_eval, targets_profit

In [13]:
df_output = pd.merge(df[['KatashikiId', 'Regist_Date', 'ReleaseDate', 'Name', 'brand_name', 'データ区分'] + features + targets_eval + targets_profit],
                     df_output.drop(columns='Kind'), on='KatashikiId', how='inner')  

In [14]:
def get_actual_pred_abs_diff(df, targets):
    for target in targets:
        df[target + '_diff'] = df[target] - df[target + '_pred']
        df[target + '_diff'] = df[target + '_diff'].abs()
    return df

from data_columns.en_jp_dictionary import *

features_en_to_jp = {}
features_en_to_jp.update(features_jp)
features_en_to_jp.update(targets_jp)
features_en_to_jp.update(targets_pred_jp)

for target in targets_eval + targets_profit:
    features_en_to_jp.update({target+'_diff': features_en_to_jp[target] + '_絶対差分'})

In [15]:
df_output = get_actual_pred_abs_diff(df_output, targets_eval + targets_profit)

In [16]:
def categorize_modellife(x):
    if x >= 0 and x <= 6:
        return 1
    elif x >= 7 and x <= 9:
        return 2
    elif x >= 10 and x <= 13:
        return 3
    elif x >= 14 and x <= 17:
        return 4
    else:
        return 5

def categorize_grossprofit(x):
    if x <= 200000:
        return 1
    elif x > 200000 and x <= 300000:
        return 2
    elif x > 300000 and x <= 400000:
        return 3
    elif x > 400000 and x <= 600000:
        return 4
    elif x > 600000 and x <= 800000:
        return 5
    else:
        return 6

for target in ['ModelLife', 'GrossProfit']:
    func = categorize_modellife if target == 'ModelLife' else categorize_grossprofit
    df_output[target+'_category' + '_pred'] = df_output[target+'_pred'].apply(lambda x : func(x))
    df_output[target+'_category'] = df_output[target].apply(lambda x : func(x))

In [17]:
df_output['output_score_sum'] = df_output[targets_eval].sum(axis=1)
df_output['output_score_pred_sum'] = df_output[[target + '_pred' for target in targets_eval]].sum(axis=1)

In [18]:
list_cols = ['KatashikiId', 'Regist_Date','ReleaseDate', 'Name',
             'brand_name',
             'データ区分', 'output_score_sum', 'output_score_pred_sum']

for target in targets_eval:
    list_cols.extend([target, target+'_pred', target+'_diff'])

for target in targets_profit:
    list_cols.extend([target, target + '_pred', target+'_diff'])

In [19]:
df_output = df_output[list_cols+features]

In [20]:
df_output = df_output.rename(columns=features_en_to_jp)
df_output = df_output.rename(columns={'Regist_Date':'登録日（年月日）', 'ReleaseDate' : 'リリース日（年月日）', 'KatashikiId':'型式ID', 'Name': '型式名', 'brand_name':'ブランド名（型式名から抽出）'})
df_output = df_output.rename(columns={'output_score_sum':'全スコア（実績）','output_score_pred_sum':'全スコア（予測）'})

In [21]:
df_output.columns.tolist()

['型式ID',
 '登録日（年月日）',
 'リリース日（年月日）',
 '型式名',
 'ブランド名（型式名から抽出）',
 'データ区分',
 '全スコア（実績）',
 '全スコア（予測）',
 '演出',
 '演出（予測値）',
 '演出_絶対差分',
 'スペック',
 'スペック（予測値）',
 'スペック_絶対差分',
 'コンテンツ',
 'コンテンツ（予測値）',
 'コンテンツ_絶対差分',
 '話題性',
 '話題性（予測値）',
 '話題性_絶対差分',
 'コンセプト性',
 'コンセプト性（予測値）',
 'コンセプト性_絶対差分',
 '時期',
 '時期（予測値）',
 '時期_絶対差分',
 '販売台数',
 '販売台数（予測値）',
 '販売台数_絶対差分',
 '稼働貢献実績',
 '稼働貢献実績（予測値）',
 '稼働貢献実績_絶対差分',
 '収益貢献実績',
 '収益貢献実績（予測値）',
 '収益貢献実績_絶対差分',
 '価格',
 '価格（予測値）',
 '価格_絶対差分',
 '粗利益',
 '粗利益（予測値）',
 '粗利益_絶対差分',
 '貢献週',
 '貢献週（予測値）',
 '貢献週_絶対差分',
 'メーカーID',
 '種類',
 '価格（円）',
 '内覧会フラグ',
 '展示会フラグ',
 '検定切れフラグ',
 '確変(突入率)',
 '確変(継続率)',
 '等価',
 '継続数平均',
 '払い出し数平均',
 '玉単価',
 'スペック名',
 'コイン単位',
 'ゲーム単位',
 'ボーナスフラグ1_1',
 'ボーナスフラグ1_2',
 'ボーナスフラグ1_3',
 'ボーナスフラグ1_4',
 'ボーナスフラグ1_5',
 'ボーナスフラグ2_1',
 'ボーナスフラグ2_2',
 'ボーナスフラグ2_3',
 'ボーナスフラグ2_4',
 'ボーナスフラグ2_5',
 'ボーナスフラグ3_1',
 'ボーナスフラグ3_2',
 'ボーナスフラグ3_3',
 'ボーナスフラグ3_4',
 'ボーナスフラグ3_5',
 'ボーナスフラグ4_1',
 'ボーナスフラグ4_2',
 'ボーナスフラグ4_3',
 'ボーナスフラグ4_4',
 'ボーナスフラグ4_5',
 'ボーナスフラグ

In [22]:
df_output

,型式ID,登録日（年月日）,リリース日（年月日）,型式名,ブランド名（型式名から抽出）,データ区分,全スコア（実績）,全スコア（予測）,演出,演出（予測値）,...,収益貢献実績（ブランド名ごと全期間平均）,価格（ブランド名ごと90日平均）,価格（ブランド名ごと365日平均）,価格（ブランド名ごと全期間平均）,粗利益（ブランド名ごと90日平均）,粗利益（ブランド名ごと365日平均）,粗利益（ブランド名ごと全期間平均）,貢献週（ブランド名ごと90日平均）,貢献週（ブランド名ごと365日平均）,貢献週（ブランド名ごと全期間平均）
0,S2019102418435101,2019-07-26 18:43:51,2020-01-05,Sパチスロ1000ちゃん PX,1000ちゃん,学習,25.00,23.90,2.5,2.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S2021040810102201,2021-01-08 10:10:22,2021-04-04,Sパチスロ1000ちゃん G1,1000ちゃん,学習対象外,0.00,24.90,NaN,2.05,...,1.5,NaN,NaN,2.5,NaN,NaN,133348.0,NaN,NaN,5.0
2,P2019100212264701,2019-07-04 12:26:47,2019-12-01,P13日の金曜日 ZB,13日の金曜日,学習,22.25,22.25,2.0,2.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P2020030410274101,2019-12-05 10:27:41,2020-05-06,P13日の金曜日 MA,13日の金曜日,学習対象外,0.00,23.10,NaN,2.10,...,1.0,NaN,3.0,3.0,NaN,389976.0,389976.0,NaN,6.0,6.0
4,P2020030410381601,2019-12-05 10:38:16,2020-05-06,P13日の金曜日 GLA,13日の金曜日,学習対象外,0.00,22.75,NaN,2.00,...,1.0,NaN,3.0,3.0,NaN,389976.0,389976.0,NaN,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2621,S2015030205560401,2014-12-02 05:56:04,2011-11-20,パチスロ「黒ひげ危機一発」,黒ひげ危機一発,学習対象外,0.00,24.80,NaN,1.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2622,S2017032318111101,2016-12-23 18:11:11,2017-05-21,パチスロ黒神 /K2,黒神,学習,21.00,23.55,2.0,2.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2623,S2015090822052701,2015-06-10 22:05:28,2015-11-01,龍が如く OF THE END ZA,龍が如く,学習,21.50,22.80,2.0,2.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2624,S2015030207474701,2014-12-02 07:47:47,2013-05-06,龍虎の拳B,龍虎の拳,学習対象外,0.00,23.30,NaN,1.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_output['登録日（年月日）'].min()

Timestamp('2014-12-02 04:43:48')

In [24]:
df_output.to_csv('analysis_notebooks/output/実績値と予測値の比較_20200101_20241127.csv', index=False, encoding='utf-8_sig')